# Recommender sytems
### Loading the libaries and reading the data

In [ ]:
pip install surprise

In [ ]:
from surprise import KNNBasic, SVD, BaselineOnly
from surprise import Dataset,accuracy
from surprise import Reader, AlgoBase
import os
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import NMF
import numpy as np

In [ ]:
data = Dataset.load_builtin('ml-100k')

### Spliting the data and training the model

In [ ]:
from surprise.model_selection import train_test_split

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
# We'll use the famous SVD algorithm.
algo = SVD()

In [ ]:
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

In [ ]:
# Then compute RMSE
accuracy.rmse(predictions)

### Cross validation

In [ ]:
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

### KNNBasic algorithm

In [ ]:
# Retrieve the trainset.
trainset = data.build_full_trainset()


In [ ]:
# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)

In [ ]:
pred = algo.test(testset)

In [ ]:
 # Compute and print Root Mean Squared Error
accuracy.rmse(predictions, verbose=True)

In [ ]:
# Compute mean absolute error
accuracy.mae(predictions, verbose=True)

### Tuning hyperparameters

In [ ]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}

In [ ]:
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)


In [ ]:
gs.fit(data)

In [ ]:
# best RMSE score
print(gs.best_score['rmse'])

In [ ]:
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

### Cosine similarity

In [ ]:
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }


In [ ]:
algo = KNNBasic(sim_options=sim_options)

In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
cosine_rmse = accuracy.rmse(predictions, verbose=True)
cosine_rmse

In [ ]:
cosine_mae = accuracy.mae(predictions, verbose=True)
cosine_mae

Splitting the data into 5 Kfolds and evaluating the performance

In [ ]:
# Run 5 k-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Hyperparameter tuning

In [ ]:
param_grid = {'bsl_options': {'method': ['als', 'sgd'],
                              'reg': [1, 2]},
              'k': [2, 3],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5],
                              'user_based': [False]}
              }

In [ ]:
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'], cv=3)


In [ ]:
gs.fit(data)


In [ ]:
# best RMSE score
tuned_cosine_rmse = gs.best_score['rmse']
print(cosine_rmse)

In [ ]:
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
tuned_cosine_mae = gs.best_score['mae']
tuned_cosine_mae

In [ ]:
# combination of parameters that gave the best RMSE score
print(gs.best_params['mae'])

In [ ]:
import pandas as pd
results_df = pd.DataFrame.from_dict(gs.cv_results)


In [ ]:
results_df.plot()

### Pearson similarity

In [ ]:
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0  # no shrinkage
               }

In [ ]:
algo = KNNBasic(sim_options=sim_options)


In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
pearson_rmse = accuracy.rmse(predictions, verbose=True)

In [ ]:
pearson_mae = accuracy.mae(predictions, verbose=True)

In [ ]:
# Run 5 k-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

**Jaccard Similarity**

In [ ]:
class jaccard(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def estimate(self, u, i):
        intersection = u & i
        jaccard = intersection/(u+i-intersection) 
        return jaccard
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
algo = BaselineOnly(bsl_options=bsl_options)


In [ ]:
algo = jaccard()

In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset, verbose=False)

In [ ]:
jaccard_rmse = accuracy.rmse(predictions, verbose=True)

In [ ]:
jaccard_mae = accuracy.mae(predictions, verbose=True)

## Triangle multiplying Jaccard

In [ ]:
class tmj_sim(AlgoBase):

    def __init__(self, sim_options={}, bsl_options={}):

        AlgoBase.__init__(self, sim_options=sim_options,
                          bsl_options=bsl_options)

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)

        # Compute baselines and similarities
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unkown.')

        # Compute similarities between u and v, where v describes all other
        # users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)
        triangle =  (1 - (u - i)/(u+i))
        intersection = u&i
        jaccard = ((intersection)/(u + i- intersection))
        tmj = triangle * jaccard
        print('The 3 nearest neighbors of user', str(u), 'are:')
        for v, sim_uv in neighbors[:3]:
            print('user {0:} with sim {1:1.2f}'.format(v, sim_uv))

        # ... Aaaaand return the baseline estimate anyway ;)
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)

In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
tmj_rmse = accuracy.rmse(predictions, verbose=True)
tmj_rmse

In [ ]:
tmj_mae = accuracy.mae(predictions, verbose=True)
tmj_mae

In [ ]:
accuracy.mse(predictions, verbose=True)

## Getting top n recommendations for each user

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [ ]:
algo = tmj_sim()

In [ ]:
algo.fit(trainset)

In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()

In [ ]:
predictions = algo.test(testset)

In [ ]:
top_n = get_top_n(predictions, n=10)

In [ ]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

## Comparison between different similarity measures

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(cosine_mae, pearson_mae, tmj_mae)

In [ ]:
print(cosine_rmse, pearson_rmse, tmj_rmse)

In [ ]:
tmj_rmse

In [ ]:
plt.plot(rmse)

In [ ]:
mae = (cosine_mae, pearson_mae,  tmj_mae)

In [ ]:
plt.plot(mae)